In [0]:
#access the external location
dbutils.fs.ls('abfss://bronze@deenucxdatabrick.dfs.core.windows.net/')

# create  external location

### Bronze

In [0]:
%sql
-- Create a location accessed using the abfss_remote_cred credential
-- https://learn.microsoft.com/en-us/azure/databricks/sql/language-manual/sql-ref-syntax-ddl-create-location

CREATE EXTERNAL LOCATION IF NOT EXISTS deenucxdatabrick_bronze 
  URL 'abfss://bronze@deenucxdatabrick.dfs.core.windows.net/'
  WITH (STORAGE CREDENTIAL `Azuredatabricks-try-credentials`);

### Silver

In [0]:
%sql
CREATE EXTERNAL LOCATION IF NOT EXISTS deenucxdatabrick_silver 
  URL 'abfss://silver@deenucxdatabrick.dfs.core.windows.net/'
  WITH (STORAGE CREDENTIAL `Azuredatabricks-try-credentials`);

### Gold

In [0]:
%sql
CREATE EXTERNAL LOCATION IF NOT EXISTS deenucxdatabrick_gold 
  URL 'abfss://gold@deenucxdatabrick.dfs.core.windows.net/'
  WITH (STORAGE CREDENTIAL `Azuredatabricks-try-credentials`);

In [0]:
%sql
DESC EXTERNAL LOCATION deenucxdatabrick_bronze;

In [0]:
%fs
ls "abfss://bronze@deenucxdatabrick.dfs.core.windows.net/"

###  Create Catalog and Schemas required for the project

1. Catalog - deen_retail (without managed location)
2. Schemas - bronze, silver and gold (with managed location)



In [0]:
%sql
DESC EXTERNAL LOCATION 

**Create catalog**

In [0]:
%sql
CREATE CATALOG  IF NOT EXISTS deen_retail;

### **Create Schemas with managed location (creates a schema in local ennvironment and pairs with adls storage)**

In [0]:
dbutils.secrets.help()

In [0]:
dbutils.secrets.listScopes()

In [0]:
%python
dbutils.secrets.list(scope="deen-databricks-retail-keyvault")

In [0]:
%python
# Set the Azure storage account credentials in the cluster's Spark configuration
spark.conf.set(
    "fs.azure.account.key.deenucxdatabrick.dfs.core.windows.net",
    dbutils.secrets.get(scope="deen-databricks-retail-keyvault", key="deen-deenucxdatabrick-key")
)

**Bronze**

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS bronze
MANAGED LOCATION "abfss://bronze@deenucxdatabrick.dfs.core.windows.net/"

**Silver**

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS silver
MANAGED LOCATION "abfss://silver@deenucxdatabrick.dfs.core.windows.net/"

**Gold**

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS gold
MANAGED LOCATION "abfss://gold@deenucxdatabrick.dfs.core.windows.net/"

In [0]:
SHOW SCHEMAS;

### Using Autoloader

In [0]:
input_file = 'abfss://bronze@deenucxdatabrick.dfs.core.windows.net/naval/autoloader/bronze'
output_path = 'abfss://bronze@deenucxdatabrick.dfs.core.windows.net/naval/autoloader/silver'


In [0]:
( spark.readStream
.format("cloudFiles")
.option("cloudFiles.format", "csv")
.option("cloudFiles.schemaLocation", f"{output_path}/autoloader/schemalocation")
.load(f"{input_file}")
.writeStream
.option("checkpointLocation", f"{output_path}/autoloader/checkpoint")
.option("path",f"{output_path}/autoloader/output")
.table("autoloader")
)